In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects

# 1. Load the data from the CSV file
try:
    df = pd.read_csv('SAR.csv')
except FileNotFoundError:
    print("Error: 'SAR.csv' not found. Please make sure the CSV file is in the same directory.")
    exit()

# 2. Prepare the data: Calculate means and SEMs
concentrations_prefixes = ['Control', 'C0.25', 'C0.5', 'C1']

# Lists to store the calculated mean and SEM column names
mean_cols = []
sem_cols = []
temp_cols_grouped = []
for prefix in concentrations_prefixes:
    current_temp_cols = [col for col in df.columns if col.startswith(prefix)]
    if current_temp_cols:
        temp_cols_grouped.append(current_temp_cols)
        mean_col_name = f'Mean_{prefix}'
        sem_col_name = f'SEM_{prefix}'
        mean_cols.append(mean_col_name)
        sem_cols.append(sem_col_name)

        
        df[mean_col_name] = df[current_temp_cols].mean(axis=1)
        df[sem_col_name] = df[current_temp_cols].sem(axis=1)
    else:
        print(f"Warning: No columns found for prefix '{prefix}'. This group will be skipped.")

# 3. Plotting
plt.figure(figsize=(7, 5))

# Define colors, markers, and labels for your specific groups

plot_colors = ['gray', 'blue', 'green', 'red']
plot_markers = ['o', 's', '^', 'v']
plot_labels = ['Control', 'Fe (0.25 mg/mL)', 'Fe (0.5 mg/mL)', 'Fe (1 mg/mL)']

# Loop through the calculated mean/SEM columns and plot
for i, mean_col in enumerate(mean_cols):
    sem_col = sem_cols[i]
    color = plot_colors[i]
    marker = plot_markers[i]
    label = plot_labels[i]

    # Plotting the error bars
    
    errorbar = plt.errorbar(df['Time(min)'].values, df[mean_col], yerr=df[sem_col],
                            color=color, linewidth=2, capsize=4, fmt='none')

    # Plotting the line and markers separately for custom effects
    plt.plot(df['Time(min)'].values, df[mean_col], color=color, linewidth=2, linestyle='-',
             marker=marker, markersize=8, label=label)

    # Apply path effects to error bar caps for better visibility
    for cap in errorbar[1]:
        cap.set_markeredgewidth(1)
        cap.set_path_effects([path_effects.Stroke(linewidth=1), path_effects.Normal()])


# Font properties
font_properties = {'fontweight': 'bold', 'fontsize': 18, 'fontfamily': 'Arial'}
number_font_properties = {'fontweight': 'bold', 'fontsize': 18, 'fontfamily': 'Arial'}
legend_font_properties = {'size': 14, 'family': 'Arial'}

# Set labels and title
plt.xlabel('Time (min)', **font_properties)
plt.ylabel('Temperature (°C)', **font_properties)

# Set x-ticks
plt.xticks(df['Time(min)'].values, ha='center', **number_font_properties)
# Set y-ticks
plt.yticks([25, 30, 35, 40, 45], **number_font_properties)
plt.xlim(left=0)

# Customize tick parameters
plt.tick_params(axis='both', which='both', length=10, width=2, labelsize=18)

# Customize spines (plot borders)
ax = plt.gca()
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Set y-axis limit
plt.ylim(bottom=25)

# Customize legend placement and properties
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), prop=legend_font_properties, frameon=False, labelspacing=1)

# Save the figure as a TIFF file with high DPI
plt.savefig('SAR_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()